In [ ]:
# Install OpenAI SDK
!pip install openai==1.84.0

# 🌤️ Manual AI Agent (No Framework): Weather Checker
This notebook demonstrates a simple AI agent without using any frameworks like LangChain.
The agent:
- Understands if a query is about weather
- Extracts the city name using OpenAI
- Returns the current weather (simulated)

In [ ]:
import httpx
from openai import OpenAI
import os
# Set your OpenAI API Key

In [ ]:
model_key = "Enter your API KEY"

In [ ]:
model_api="https://llama-3-1-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1"
model="meta-llama/Llama-3.1-8B-Instruct"



client = OpenAI(
    base_url=model_api,
    api_key=model_key,
    http_client=httpx.Client(verify=False)  # DO NOT DO THIS IN PRODUCTION!!!!
)

In [2]:
# Dummy function to simulate weather API
def get_weather(city):
    return f"The current weather in {city} is 28°C, partly cloudy."

In [3]:
# Function to decide if query is about weather
def decide_action(user_input: str) -> str:
    """
    Classifies whether the user query is weather-related.

    Returns
    -------
    str
        'weather' if the LLM says the query is about the weather,
        otherwise 'unknown'.
    """

    print("\n🔎  Classifying query intent...")

    # Ask the LLM for a one-word verdict.
    prompt = f"Does this query ask about weather? '{user_input}' Answer yes or no."


    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=5,           # We only need 'yes' or 'no'
    )

    verdict = response.choices[0].message.content.strip().lower()
    print(f"🧠  LLM verdict: {verdict}")

    return "weather" if verdict.startswith("yes") else "unknown"


In [4]:
def run_agent(user_input):
    """
    Full agent flow:
    - Detects the type of query (weather or not)
    - Extracts city name if needed
    - Fetches weather data (simulated)
    - Asks LLM to summarize the result
    """

    print(f"\n🟡 Received query: '{user_input}'")

    # Step 1: Let LLM decide if it's a weather-related query
    task = decide_action(user_input)
    print(f"🔍 LLM classification: '{task}'")

    if task == 'weather':
        # Step 2: Ask LLM to extract the city name from the query
        city_prompt = f"From this query, identify the city name only: '{user_input}'"
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": city_prompt}]
        )
        city = response.choices[0].message.content.strip()
        print(f"🏙️ Extracted city: {city}")

        # Step 3: Simulate fetching weather from an API
        weather_data = get_weather(city)
        print(f"🌤️ Raw weather data: {weather_data}")

        # Step 4: Ask LLM to summarize the raw weather data
        summary_prompt = f"Given this raw weather information from a weather API, rephrase it into a natural and friendly sentence: '{weather_data}'"
        summary_response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": summary_prompt}]
        )
        final_response = summary_response.choices[0].message.content.strip()
        print(f"✅ Final summarized response: {final_response}")
        return final_response

    else:
        print("⚠️ Query is not weather-related.")
        return "Sorry, I can only help with weather-related questions right now."


In [6]:
# Test the manual agent
query = "What's the score of the match which happened in Bangalore today?"
print(run_agent(query))


🟡 Received query: 'What's the score of the match which happened in Bangalore today?'

🔎  Classifying query intent...
🧠  LLM verdict: no
🔍 LLM classification: 'unknown'
⚠️ Query is not weather-related.
Sorry, I can only help with weather-related questions right now.
